In [ ]:
!pip install selenium

In [4]:
from selenium import webdriver #permite criar o navegador
from selenium.webdriver.common.keys import Keys #permite escrever no navegador
from selenium.webdriver.common.by import By #permite selecionar itens no navegador

navegador = webdriver.Chrome()

# Passo 1: pegar a cotação do dólar
navegador.get('https://www.google.com.br/')

#pesquisar no google por "cotação dólar"
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#pegar a cotação no google
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

In [ ]:
# Passo 2: pegar a cotação do euro

In [ ]:
# Passo 3: pegar a cotação do ouro

In [ ]:
# Passo 4: atualizar a base de preços (atualizando o preço de compra e o de venda)

In [ ]:
# Passo 5: exportar a base de preços atualizado